# Assignment - POS tagging using modified Viterbi

## Section-1: Vanilla Viterbi algorithm without dealing unknown words

### Data Preparation

In [1]:
#Importing libraries
import nltk, re, pprint
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
# first few tagged sentences
print(nltk_data[:5])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [4]:
# splitting into train and test or validation (95% to 5% ratio considered here)
random.seed(1234)
train_set, test_set = train_test_split(nltk_data, test_size = 0.05)

print(len(train_set))
print(len(test_set))
print(train_set[:15])

3718
196
[[('Prosecutors', 'NOUN'), ('alleged', 'VERB'), ('that', 'ADP'), ('she', 'PRON'), ('was', 'VERB'), ('trying', 'VERB'), ('*-1', 'X'), ('to', 'PRT'), ('bolster', 'VERB'), ('students', 'NOUN'), ("'", 'PRT'), ('scores', 'NOUN'), ('*-1', 'X'), ('to', 'PRT'), ('win', 'VERB'), ('a', 'DET'), ('bonus', 'NOUN'), ('under', 'ADP'), ('the', 'DET'), ('state', 'NOUN'), ("'s", 'PRT'), ('1984', 'NUM'), ('Education', 'NOUN'), ('Improvement', 'NOUN'), ('Act', 'NOUN'), ('.', '.')], [('``', '.'), ('Lighthouse', 'NOUN'), ('II', 'NOUN'), ("''", '.'), ('was', 'VERB'), ('painted', 'VERB'), ('*-1', 'X'), ('in', 'ADP'), ('oils', 'NOUN'), ('by', 'ADP'), ('the', 'DET'), ('playwright', 'NOUN'), ('in', 'ADP'), ('1901', 'NUM'), ('...', '.')], [('``', '.'), ('It', 'PRON'), ('is', 'VERB'), ('these', 'DET'), ('645,000', 'NUM'), ('tons', 'NOUN'), ('that', 'DET'), ('*T*-1', 'X'), ('are', 'VERB'), ('in', 'ADP'), ('question', 'NOUN'), ('for', 'ADP'), ('this', 'DET'), ('crop', 'NOUN'), ('year', 'NOUN'), (',', '.'), 

In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95672

In [6]:
# tokens or words in train set 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['Prosecutors',
 'alleged',
 'that',
 'she',
 'was',
 'trying',
 '*-1',
 'to',
 'bolster',
 'students']

In [7]:
# vocabulary of unique words in train_set
V = set(tokens)
print(len(V))

12088


In [8]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [9]:
print(set(T))

{'ADJ', '.', 'PRT', 'ADP', 'CONJ', 'X', 'DET', 'VERB', 'PRON', 'NOUN', 'NUM', 'ADV'}


### Building HMM

#### Emission probabilities 

In [10]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [11]:
w_given_t

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# compute word given tag, P(W/T)
def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1] == tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0] == word]
    count_w_given_tag = len(w_given_tag_list)
    return (count_w_given_tag, count_tag)   

In [13]:
print(word_given_tag('investors', 'NOUN'))
print(word_given_tag('investors', 'VERB'))

(60, 27425)
(0, 12866)


#### Transition probabilities

In [14]:
# compute tag2 (t2) given tag1 (t1)
def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t == t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index] == t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [15]:
# check 
print(t2_given_t1(t2 = 'NOUN', t1 = 'VERB'))
print(t2_given_t1(t2 = '.', t1 = 'VERB'))

(1425, 12866)
(451, 12866)


In [16]:
# P(tag/start)
print(t2_given_t1(t2 = 'NOUN', t1 = '.'))
print(t2_given_t1(t2 = 'VERB', t1 = '.'))
print(t2_given_t1(t2 = 'CONJ', t1 = '.'))

(2490, 11169)
(985, 11169)
(646, 11169)


In [17]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [18]:
tags_matrix # with transition probabilities

array([[6.58456460e-02, 6.65024593e-02, 1.06732352e-02, 7.81609192e-02,
        1.70771759e-02, 2.05254517e-02, 4.76190494e-03, 1.18226605e-02,
        6.56814431e-04, 6.98193789e-01, 2.10180618e-02, 4.76190494e-03],
       [4.46772315e-02, 9.33834687e-02, 2.41740537e-03, 9.12346691e-02,
        5.78386597e-02, 2.74867937e-02, 1.71456710e-01, 8.81905258e-02,
        6.62548095e-02, 2.22938493e-01, 8.18336457e-02, 5.21980487e-02],
       [8.58355090e-02, 4.40600514e-02, 1.95822446e-03, 2.05613580e-02,
        2.28459528e-03, 1.37075717e-02, 1.02480419e-01, 3.99151444e-01,
        1.79503914e-02, 2.45757177e-01, 5.58094010e-02, 1.04438644e-02],
       [1.07371792e-01, 4.01709415e-02, 1.49572652e-03, 1.72008555e-02,
        7.47863261e-04, 3.53632495e-02, 3.20192307e-01, 8.54700897e-03,
        6.98717982e-02, 3.22542727e-01, 6.30341917e-02, 1.34615386e-02],
       [1.17266849e-01, 3.60110812e-02, 4.61680535e-03, 5.17082177e-02,
        4.61680524e-04, 8.31024908e-03, 1.19575255e-01, 1.57

In [19]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [20]:
tags_df

,ADJ,.,PRT,ADP,CONJ,X,DET,VERB,PRON,NOUN,NUM,ADV
ADJ,0.065846,0.066502,0.010673,0.078161,0.017077,0.020525,0.004762,0.011823,0.000657,0.698194,0.021018,0.004762
.,0.044677,0.093383,0.002417,0.091235,0.057839,0.027487,0.171457,0.088191,0.066255,0.222938,0.081834,0.052198
PRT,0.085836,0.044060,0.001958,0.020561,0.002285,0.013708,0.102480,0.399151,0.017950,0.245757,0.055809,0.010444
ADP,0.107372,0.040171,0.001496,0.017201,0.000748,0.035363,0.320192,0.008547,0.069872,0.322543,0.063034,0.013462
CONJ,0.117267,0.036011,0.004617,0.051708,0.000462,0.008310,0.119575,0.157895,0.058633,0.348107,0.042013,0.055402
X,0.016858,0.166349,0.183683,0.143289,0.010655,0.074587,0.054071,0.204198,0.054389,0.062977,0.002704,0.026240
DET,0.205328,0.018124,0.000243,0.009366,0.000487,0.044763,0.005109,0.039898,0.003528,0.638608,0.021774,0.012772
VERB,0.065444,0.035054,0.031711,0.090160,0.005441,0.217317,0.134929,0.168273,0.035675,0.110757,0.023162,0.082077
PRON,0.072769,0.039831,0.012256,0.022980,0.005362,0.093451,0.009575,0.485255,0.008043,0.209881,0.006894,0.033704
NOUN,0.012069,0.239781,0.044339,0.177174,0.042844,0.029025,0.012835,0.146363,0.004667,0.264540,0.009298,0.017065


In [21]:
tags_df.loc['.', :]  # transition probabilities for starting tags

ADJ     0.044677
.       0.093383
PRT     0.002417
ADP     0.091235
CONJ    0.057839
X       0.027487
DET     0.171457
VERB    0.088191
PRON    0.066255
NOUN    0.222938
NUM     0.081834
ADV     0.052198
Name: ., dtype: float32

### Build the vanilla Viterbi based POS tagger

In [22]:
len(train_tagged_words)

95672

In [23]:
# Viterbi Heuristic
def Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))


### Evaluating on Validation set

In [24]:
# Let's test our Viterbi algorithm on a few sample sentences of validation dataset

random.seed(1234)

# list of sents
test_run = [test_set[i] for i in range(len(test_set))]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

[[('Norman', 'NOUN'),
  ('Ricken', 'NOUN'),
  (',', '.'),
  ('52', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('president', 'NOUN'),
  ('and', 'CONJ'),
  ('chief', 'NOUN'),
  ('operating', 'VERB'),
  ('officer', 'NOUN'),
  ('of', 'ADP'),
  ('Toys', 'NOUN'),
  ('``', '.'),
  ('R', 'NOUN'),
  ("''", '.'),
  ('Us', 'NOUN'),
  ('Inc.', 'NOUN'),
  (',', '.'),
  ('and', 'CONJ'),
  ('Frederick', 'NOUN'),
  ('Deane', 'NOUN'),
  ('Jr.', 'NOUN'),
  (',', '.'),
  ('63', 'NUM'),
  (',', '.'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Signet', 'NOUN'),
  ('Banking', 'NOUN'),
  ('Corp.', 'NOUN'),
  (',', '.'),
  ('were', 'VERB'),
  ('elected', 'VERB'),
  ('*-15', 'X'),
  ('directors', 'NOUN'),
  ('of', 'ADP'),
  ('this', 'DET'),
  ('consumer', 'NOUN'),
  ('electronics', 'NOUN'),
  ('and', 'CONJ'),
  ('appliances', 'NOUN'),
  ('retailing', 'NOUN'),
  ('chain', 'NOUN'),
  ('.', '.')],
 [('The', 'DET'),
  ('court', 'NOUN'),
  ('hearing', 'NOUN'),
  ('beg

In [25]:
print(len(test_tagged_words))

5004


In [26]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [27]:
print("Time taken in seconds: ", difference)
print(tagged_seq)

Time taken in seconds:  1918.5968923568726
[('Norman', 'NOUN'), ('Ricken', 'ADJ'), (',', '.'), ('52', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('president', 'NOUN'), ('and', 'CONJ'), ('chief', 'NOUN'), ('operating', 'NOUN'), ('officer', 'NOUN'), ('of', 'ADP'), ('Toys', 'ADJ'), ('``', '.'), ('R', 'ADJ'), ("''", '.'), ('Us', 'ADJ'), ('Inc.', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('Frederick', 'NOUN'), ('Deane', 'ADJ'), ('Jr.', 'NOUN'), (',', '.'), ('63', 'NUM'), (',', '.'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Signet', 'ADJ'), ('Banking', 'NOUN'), ('Corp.', 'NOUN'), (',', '.'), ('were', 'VERB'), ('elected', 'VERB'), ('*-15', 'X'), ('directors', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('consumer', 'NOUN'), ('electronics', 'NOUN'), ('and', 'CONJ'), ('appliances', 'NOUN'), ('retailing', 'NOUN'), ('chain', 'NOUN'), ('.', '.'), ('The', 'DET'), ('court', 'NOUN'), ('hearing', 'NOUN'), ('began', 'VERB'), ('in', 'ADP'), ('early', 'ADJ'), ('October', 'NOUN'

In [28]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [29]:
accuracy = len(check)/len(tagged_seq)

In [30]:
round(accuracy*100,2)

91.89

Universal Treebank tagging accuracy on validation data with Vanilla Vierbi Heuristic algorithm is found to be 92% without dealing any unknown words.

In [31]:
print(test_run_base)

[('Norman', 'NOUN'), ('Ricken', 'NOUN'), (',', '.'), ('52', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('president', 'NOUN'), ('and', 'CONJ'), ('chief', 'NOUN'), ('operating', 'VERB'), ('officer', 'NOUN'), ('of', 'ADP'), ('Toys', 'NOUN'), ('``', '.'), ('R', 'NOUN'), ("''", '.'), ('Us', 'NOUN'), ('Inc.', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('Frederick', 'NOUN'), ('Deane', 'NOUN'), ('Jr.', 'NOUN'), (',', '.'), ('63', 'NUM'), (',', '.'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Signet', 'NOUN'), ('Banking', 'NOUN'), ('Corp.', 'NOUN'), (',', '.'), ('were', 'VERB'), ('elected', 'VERB'), ('*-15', 'X'), ('directors', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('consumer', 'NOUN'), ('electronics', 'NOUN'), ('and', 'CONJ'), ('appliances', 'NOUN'), ('retailing', 'NOUN'), ('chain', 'NOUN'), ('.', '.'), ('The', 'DET'), ('court', 'NOUN'), ('hearing', 'NOUN'), ('began', 'VERB'), ('in', 'ADP'), ('early', 'ADJ'), ('October', 'NOUN'), ('at', 'ADP'), ('the', 'DET'), ('r

In [32]:
incorrect_tagged_cases = [[test_run_base[i-1], j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]

In [33]:
incorrect_tagged_cases

[[('Norman', 'NOUN'), (('Ricken', 'ADJ'), ('Ricken', 'NOUN'))],
 [('chief', 'NOUN'), (('operating', 'NOUN'), ('operating', 'VERB'))],
 [('of', 'ADP'), (('Toys', 'ADJ'), ('Toys', 'NOUN'))],
 [('``', '.'), (('R', 'ADJ'), ('R', 'NOUN'))],
 [("''", '.'), (('Us', 'ADJ'), ('Us', 'NOUN'))],
 [('Frederick', 'NOUN'), (('Deane', 'ADJ'), ('Deane', 'NOUN'))],
 [('of', 'ADP'), (('Signet', 'ADJ'), ('Signet', 'NOUN'))],
 [('Anthony', 'NOUN'), (('Hazell', 'ADJ'), ('Hazell', 'NOUN'))],
 [('district', 'NOUN'), (('auditor', 'ADJ'), ('auditor', 'NOUN'))],
 [("n't", 'ADV'), (('vested', 'ADJ'), ('vested', 'VERB'))],
 [('strong', 'ADJ'), (('livestock', 'ADJ'), ('livestock', 'NOUN'))],
 [("'s", 'PRT'), (('cattle', 'ADJ'), ('cattle', 'NOUN'))],
 [('cattle', 'NOUN'), (('inventory', 'ADJ'), ('inventory', 'NOUN'))],
 [('dropped', 'VERB'), (('close', 'VERB'), ('close', 'ADV'))],
 [('30-year', 'ADJ'), (('low', 'ADJ'), ('low', 'NOUN'))],
 [('bought', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))],
 [('$', '.'), (('701', '

In [34]:
# Percentage of incorrect tags
round(len(incorrect_tagged_cases)/len(tagged_seq)*100,2)

8.11

For unknown words (means words present in the validation_set but not in the train_set), emission probability in Viterbi algorithm becomes zero and algorithm chooses the 1st tag arbitrarily. This is the limitation with the above Viterbi algorithm.

### Tagging accuracy check of the sample test sentences (provided in the assignment) with Vanilla Viterbi

In [35]:
## Testing with sample test sentences provided in the assignment
sentence_test = "Android is a mobile operating system developed by Google. Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013. Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose. Twitter is an online news and social networking service on which users post and interact with messages known as tweets. Before entering politics, Donald Trump was a domineering businessman and a television personality. The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years. This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe. Show me the cheapest round trips from Dallas to Atlanta I would like to see flights from Denver to Philadelphia. Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco. NASA invited social media users to experience the launch of ICESAT-2 Satellite."
print(sentence_test)

Android is a mobile operating system developed by Google. Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013. Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose. Twitter is an online news and social networking service on which users post and interact with messages known as tweets. Before entering politics, Donald Trump was a domineering businessman and a television personality. The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years. This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe. Show me the cheapest round trips from Dallas to Atlanta I would like to see flights from Denver to Philadelphia. Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco. NASA invited social media users to experience the launch of ICESAT-2 Satellite.


In [36]:
words_sample_test = word_tokenize(sentence_test)
words_sample_test[:5]

['Android', 'is', 'a', 'mobile', 'operating']

In [37]:
len(words_sample_test)

181

In [38]:
# nltk tagging for words in sample test sentences
tagged_sample_test = nltk.pos_tag(words_sample_test, tagset='universal')
tagged_sample_test[:5]

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN')]

In [39]:
# Viterbi tagging for words in sample test sentences

start = time.time()
viterbi_tagged_sample_test = Viterbi(words_sample_test)
end = time.time()
difference = end-start

In [40]:
print("Time taken in seconds: ", difference)
print(viterbi_tagged_sample_test)

Time taken in seconds:  43.12677597999573
[('Android', 'ADJ'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'ADJ'), ('.', '.'), ('Android', 'ADJ'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'ADJ'), ('worldwide', 'ADJ'), ('on', 'ADP'), ('smartphones', 'ADJ'), ('since', 'ADP'), ('2011', 'ADJ'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'ADJ'), ('.', '.'), ('Google', 'ADJ'), ('and', 'CONJ'), ('Twitter', 'ADJ'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'ADJ'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'ADJ'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'ADJ'), ("'s", 'PRT'), ('firehose', 'ADJ'), ('.', '.'), ('Twitter', 'ADJ'), ('is', 'VERB'), ('an', 'DET'), ('online', 'ADJ'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on'

In [41]:
# accuracy with original viterbi
check_ovtb = [i for i, j in zip(viterbi_tagged_sample_test, tagged_sample_test) if i == j] 

In [42]:
accuracy_ovtb = len(check_ovtb)/len(viterbi_tagged_sample_test)

In [43]:
round(accuracy_ovtb*100,2)

77.35

With original viterbi, tagging accuracy on the sample test sentences are 77.4% only.

### End of Section-1

## Section-2: Handelling unknown words

Here unknown words are those words present in validation set (or in nltk test set) and in sample test sentences, but not in the train_set.

### Solve the problem of unknown words

### Solution1 - Probability based solution - Following logic can be incorporated in the Viterbi
1. For words in the corpus (i.e. in train_bag), state probability = transition probability * emission probability
2. For words not in the corpus (i.e. not in train_bag), state probability = transition probability 

In [44]:
# Improved Viterbi Heuristic
def Viterbi_improved1(words, train_bag = train_tagged_words):
    state = []
    T1 = list(set([pair[1] for pair in train_bag]))  # for tags in train_bag
    T2 = list(set([pair[0] for pair in train_bag]))  # for words in train_bag
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T1:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                         
        # compute emission and state probabilities
            if word in T2:
                emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability = emission_p * transition_p
            else:        
                state_probability = transition_p
            p.append(state_probability)            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T1[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [45]:
# checking the tagging on validation set with improved viterbi 
start = time.time()
tagged_seq_improved1 = Viterbi_improved1(test_tagged_words)
end = time.time()
difference = end-start

In [46]:
print("Time taken in seconds: ", difference)
print(tagged_seq_improved1)

Time taken in seconds:  1093.2659747600555
[('Norman', 'NOUN'), ('Ricken', 'NOUN'), (',', '.'), ('52', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('president', 'NOUN'), ('and', 'CONJ'), ('chief', 'NOUN'), ('operating', 'NOUN'), ('officer', 'NOUN'), ('of', 'ADP'), ('Toys', 'NOUN'), ('``', '.'), ('R', 'NOUN'), ("''", '.'), ('Us', 'NOUN'), ('Inc.', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('Frederick', 'NOUN'), ('Deane', 'NOUN'), ('Jr.', 'NOUN'), (',', '.'), ('63', 'NUM'), (',', '.'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Signet', 'NOUN'), ('Banking', 'NOUN'), ('Corp.', 'NOUN'), (',', '.'), ('were', 'VERB'), ('elected', 'VERB'), ('*-15', 'X'), ('directors', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('consumer', 'NOUN'), ('electronics', 'NOUN'), ('and', 'CONJ'), ('appliances', 'NOUN'), ('retailing', 'NOUN'), ('chain', 'NOUN'), ('.', '.'), ('The', 'DET'), ('court', 'NOUN'), ('hearing', 'NOUN'), ('began', 'VERB'), ('in', 'ADP'), ('early', 'ADJ'), ('October', 

In [47]:
# accuracy with improved Viterbi
check_improved1 = [i for i, j in zip(tagged_seq_improved1, test_run_base) if i == j] 

In [48]:
accuracy_improved1 = len(check_improved1)/len(tagged_seq_improved1)

In [49]:
round(accuracy_improved1*100,2)

94.74

In [50]:
incorrect_tagged_cases_1 = [[test_run_base[i-1], j] for i, j in enumerate(zip(tagged_seq_improved1, test_run_base)) if j[0]!=j[1]]

In [51]:
incorrect_tagged_cases_1

[[('chief', 'NOUN'), (('operating', 'NOUN'), ('operating', 'VERB'))],
 [("'s", 'PRT'), (('cattle', 'VERB'), ('cattle', 'NOUN'))],
 [('cattle', 'NOUN'), (('inventory', 'X'), ('inventory', 'NOUN'))],
 [('dropped', 'VERB'), (('close', 'VERB'), ('close', 'ADV'))],
 [('30-year', 'ADJ'), (('low', 'ADJ'), ('low', 'NOUN'))],
 [('bought', 'VERB'), (('up', 'ADV'), ('up', 'PRT'))],
 [('mortgage', 'NOUN'),
  (('securities-based', 'NOUN'), ('securities-based', 'ADJ'))],
 [('$', '.'), (('701', 'NOUN'), ('701', 'NUM'))],
 [('The', 'DET'), (('assistant', 'NOUN'), ('assistant', 'ADJ'))],
 [('have', 'VERB'), (('AIDS', 'X'), ('AIDS', 'NOUN'))],
 [('by', 'ADP'), (('about', 'ADP'), ('about', 'ADV'))],
 [('to', 'PRT'), (('8.75', 'VERB'), ('8.75', 'NUM'))],
 [('has', 'VERB'), (('shown', 'X'), ('shown', 'VERB'))],
 [('before', 'ADP'), (('that', 'DET'), ('that', 'ADP'))],
 [('debt', 'NOUN'), (('forgiven', 'NOUN'), ('forgiven', 'VERB'))],
 [('Beijing', 'NOUN'), (('food-shop', 'NOUN'), ('food-shop', 'ADJ'))],
 [

In [52]:
# Percentage of incorrect tags now with improved viterbi
round(len(incorrect_tagged_cases_1)/len(tagged_seq_improved1)*100,2)

5.26

#### Thus with solution-1 probability based improvement in Viterbi algorithm, tagging accuracy increased from 92% to 95% on the validation set.

### Tagging accuracy check of the sample test sentences (provided in the assignment) with 1st improved Viterbi

In [53]:
# 1st improved Viterbi tagging for words in sample test sentences

start = time.time()
viterbi1_tagged_sample_test = Viterbi_improved1(words_sample_test)
end = time.time()
difference = end-start

In [54]:
print("Time taken in seconds: ", difference)
print(viterbi1_tagged_sample_test)

Time taken in seconds:  28.10514187812805
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NOUN'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NOUN'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NOUN'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'X'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'VERB'), ("'s", 'PRT'), ('firehose', 'VERB'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', '

In [55]:
# accuracy with 1st improved Viterbi
check_vtb1 = [i for i, j in zip(viterbi1_tagged_sample_test, tagged_sample_test) if i == j] 

In [56]:
accuracy_vtb1 = len(check_vtb1)/len(viterbi1_tagged_sample_test)

In [57]:
round(accuracy_vtb1*100,2)

89.5

#### Thus with 1st improved viterbi, tagging accuracy increased from 77.4% (for original viterbi) to 89.5% on the sample test sentences.

List of few words having tagging with Vanilla viterbi vs. 1st improved vitrbi -

1. ('Android', 'ADJ') with Vanilla, ('Android', 'NOUN') with 1st improved
2. ('Google', 'ADJ') with Vanilla, ('Google', 'NOUN') with 1st improved
3. ('smartphones', 'ADJ') with Vanilla, ('smartphones', 'NOUN') with 1st improved
4. ('Twitter', 'ADJ') with Vanilla, ('Twitter', 'NOUN') with 1st improved
5. ('online', 'ADJ') with Vanilla, ('online', 'NOUN') with 1st improved
6. ('messages', 'ADJ') with Vanilla, ('messages', 'NOUN') with 1st improved
7. ('tweets', 'ADJ') with Vanilla, ('tweets', 'NOUN') with 1st improved
8. ('FIFA', 'ADJ') with Vanilla, ('FIFA', 'NOUN') with 1st improved

### Solution2 - Rule based improvement - Following logic can be incorporated along with the original Viterbi 
1. Assign all unknown words the most frequent POS tag of the corpus (nltk_data)
2. Tagging based on morphological feature e.g. -ing or -ed is the ending letters of a VB

In [58]:
# Finding the number of unique POS tags in the corpus

total_tagged_words = [tup for sent in nltk_data for tup in sent]
total_tags = [pair[1] for pair in total_tagged_words]
unique_tags = set(total_tags)
len(unique_tags)

12

In [59]:
# Finding the most frequent tag in the corpus

from collections import Counter
tag_counts = Counter(total_tags)
tag_counts

Counter({'NOUN': 28867,
         '.': 11715,
         'NUM': 3546,
         'ADJ': 6397,
         'VERB': 13564,
         'DET': 8725,
         'ADP': 9857,
         'CONJ': 2265,
         'X': 6613,
         'ADV': 3171,
         'PRT': 3219,
         'PRON': 2737})

In [60]:
# the most common tags can be seen using the most_common() method of Counter
tag_counts.most_common(12)

[('NOUN', 28867),
 ('VERB', 13564),
 ('.', 11715),
 ('ADP', 9857),
 ('DET', 8725),
 ('X', 6613),
 ('ADJ', 6397),
 ('NUM', 3546),
 ('PRT', 3219),
 ('ADV', 3171),
 ('PRON', 2737),
 ('CONJ', 2265)]

#### Noun is the most frequent tag in this universal nltk data. 

In [61]:
# vocabulary of unique words in train_set
V = set(tokens)
len(V)

12088

In [62]:
# vocabulary of unique words in validation_set
V_validation = set(test_tagged_words)
len(V_validation)

1848

common words between train_set and validation_set are the known words, remaining words in the validation set are unknown and to be found out now.

In [63]:
list(V_validation)

['fluctuations',
 'manufacturer',
 'employed',
 'good',
 'zinc',
 'Yesterday',
 'Puerto',
 'share',
 'student',
 'demonstrators',
 'thinking',
 'bombarding',
 'expenses',
 "'re",
 'sign',
 'goods',
 'declined',
 '*T*-63',
 'Duke',
 'robotic',
 'guilders',
 'purpose',
 '50',
 'Nekoosa',
 'against',
 'since',
 'provisions',
 'carbon',
 'PRODUCTS',
 'eliminates',
 'senior',
 '456.64',
 'suppliers',
 'consumer',
 'Herald',
 'centralized',
 'His',
 'decision',
 'Wedtech',
 'editor',
 'Finland',
 'half',
 '1988',
 'including',
 'is',
 'McGovern',
 'bellringers',
 'disappears',
 'emerged',
 'Also',
 'emigres',
 'managers',
 'yen-support',
 'Stock',
 'Not',
 'plan',
 'constitutional',
 'easy',
 'of',
 'existence',
 'stages',
 'Bumkins',
 'Committee',
 '*T*-80',
 'competition',
 'exceed',
 'Jr.',
 'designers',
 'fast-food',
 'crash',
 'ringer',
 'indefinitely',
 'Larry',
 'shipbuilding',
 'Institution',
 'acceptance',
 'Japanese',
 'Mariotta',
 'spend',
 'rapprochement',
 'decline',
 'unloaded'

In [64]:
# function to find difference of two lists
def list_diff(list1, list2): 
    return (list(list1 - list2)) 

In [65]:
# for unknown words in validation set
unknown_word_validation = list_diff(V_validation, V)
unknown_word_validation

['fluctuations',
 'definitive',
 'zinc',
 'Puerto',
 'assembly-line',
 'Bowes',
 'bombarding',
 'inventory',
 'apples',
 '1.39',
 'Duke',
 'morale',
 'robotic',
 'Preferences',
 '*T*-246',
 'academics',
 'carbon',
 'PRODUCTS',
 'altar',
 '456.64',
 'circle',
 'centralized',
 'Coincident',
 'signaling',
 '701',
 'bellringers',
 'disappears',
 '446.62',
 'overpriced',
 'emerged',
 'emigres',
 'yen-support',
 'muscling',
 'provoked',
 'nonfinancial',
 'reportedly',
 'existence',
 'shown',
 'Bumkins',
 'glamorize',
 'colder',
 'Hazell',
 'Derek',
 'involvement',
 'postponed',
 'Alstyne',
 '644',
 'U.S.-Japan',
 'rapprochement',
 '8.75',
 'unloaded',
 'inhibit',
 'Reuter',
 'screened',
 'contrasts',
 'swapping',
 'foot',
 'crookery',
 'bled',
 'revolt',
 'Generalized',
 'fawning',
 'Reached',
 'Nigel',
 'wanting',
 '18-a-share',
 'EST',
 'imposing',
 '*T*-225',
 'weight',
 'reminded',
 'Doerflinger',
 'Advancing',
 'arsenide',
 'intervention',
 'craft',
 'Irwin',
 'Team',
 'skepticism',
 'b

In [66]:
len(unknown_word_validation) # number of unknown words in validation set 

320

In [67]:
# known words in validation set
known_word_validation = list_diff(V_validation, set(unknown_word_validation))

In [68]:
len(known_word_validation) # number of known words in validation set 

1528

In [69]:
# from this unknown words list, I will separate out words ending with 'ed', 'ing', 's', 'es' having morphological features
ed_verbs = [word for word in unknown_word_validation if word.endswith('ed')]
ing_verbs = [word for word in unknown_word_validation if word.endswith('ing')]
s_verbs = [word for word in unknown_word_validation if word.endswith('s')]
es_verbs = [word for word in unknown_word_validation if word.endswith('es')]

In [70]:
# list of morphological unknown words
morph_unknown_word_validation = ed_verbs + ing_verbs + s_verbs + es_verbs

In [71]:
morph_unknown_word_validation[:5]

['centralized', 'overpriced', 'emerged', 'provoked', 'postponed']

In [72]:
len(morph_unknown_word_validation)

126

In [73]:
print(type(morph_unknown_word_validation))

<class 'list'>


In [74]:
remain_unknown_word_validation = list_diff(set(unknown_word_validation), set(morph_unknown_word_validation))

So finally there are two category of unknown words in validation set, one is morphological and second is remaining unknown words.

In [75]:
# tagging remaining unknown words in validation set with nltk universal tag sets
tagged_remain_unknown_words_validation = nltk.pos_tag(remain_unknown_word_validation, tagset='universal')

In [76]:
tagged_remain_unknown_words_validation

[('arsenide', 'ADV'),
 ('cozy', 'ADJ'),
 ('intervention', 'NOUN'),
 ('847', 'NUM'),
 ('born', 'VERB'),
 ('craft', 'NOUN'),
 ('AIDS', 'NOUN'),
 ('definitive', 'ADJ'),
 ('Irwin', 'NOUN'),
 ('NESB', 'NOUN'),
 ('bribery', 'NOUN'),
 ('Team', 'NOUN'),
 ('skepticism', 'NOUN'),
 ('zinc', 'NOUN'),
 ('akin', 'ADJ'),
 ('forgiven', 'ADV'),
 ('Puerto', 'NOUN'),
 ('architectural', 'ADJ'),
 ('free-lance', 'NOUN'),
 ('livestock', 'NOUN'),
 ('assembly-line', 'ADJ'),
 ('GOODY', 'NOUN'),
 ('59-year-old', 'ADJ'),
 ('inventory', 'NOUN'),
 ('waiver', 'NOUN'),
 ('Adolph', 'NOUN'),
 ('1.39', 'NUM'),
 ('please', 'NOUN'),
 ('surgeon', 'NOUN'),
 ('Duke', 'NOUN'),
 ('morale', 'NOUN'),
 ('plane', 'NOUN'),
 ('robotic', 'ADJ'),
 ('palace', 'NOUN'),
 ('cattle', 'NOUN'),
 ('Norfolk', 'NOUN'),
 ('hole', 'VERB'),
 ('*T*-246', 'ADJ'),
 ('insider', 'NOUN'),
 ('14.00', 'NUM'),
 ('*T*-154', 'ADJ'),
 ('advertorial', 'ADJ'),
 ('limbo', 'NOUN'),
 ('carbon', 'NOUN'),
 ('sex', 'NOUN'),
 ('PRODUCTS', 'NOUN'),
 ('altar', 'ADJ'),
 

In [77]:
total_tags_remain_uwds = [pair[1] for pair in tagged_remain_unknown_words_validation] # uwds stand for remaining unknown words

In [78]:
tag_counts_remain_uwds = Counter(total_tags_remain_uwds)

In [79]:
tag_counts_remain_uwds.most_common(3)

[('NOUN', 122), ('ADJ', 49), ('NUM', 29)]

Unknown words are mostly tagged with nouns already.

In [80]:
# specify patterns for tagging morphological features
patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),               # past tense
    (r'.*s$', 'NOUN'),                # plural nouns
    (r'.*es$', 'VERB')                # 3rd singular present
]

In [81]:
# using Regex rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [82]:
tagged_morph_unknown_word_validation = rule_based_tagger.tag(morph_unknown_word_validation)
tagged_morph_unknown_word_validation[:3]

[('centralized', 'VERB'), ('overpriced', 'VERB'), ('emerged', 'VERB')]

In [83]:
len(tagged_morph_unknown_word_validation)

126

In [84]:
tagged_unknown_word_validation = tagged_morph_unknown_word_validation + tagged_remain_unknown_words_validation
tagged_unknown_word_validation

[('centralized', 'VERB'),
 ('overpriced', 'VERB'),
 ('emerged', 'VERB'),
 ('provoked', 'VERB'),
 ('postponed', 'VERB'),
 ('unloaded', 'VERB'),
 ('screened', 'VERB'),
 ('bled', 'VERB'),
 ('Generalized', 'VERB'),
 ('Reached', 'VERB'),
 ('reminded', 'VERB'),
 ('cushioned', 'VERB'),
 ('unfettered', 'VERB'),
 ('fashioned', 'VERB'),
 ('unconsolidated', 'VERB'),
 ('booked', 'VERB'),
 ('sputtered', 'VERB'),
 ('vested', 'VERB'),
 ('exuded', 'VERB'),
 ('shirt-sleeved', 'VERB'),
 ('hunted', 'VERB'),
 ('battered', 'VERB'),
 ('stabbed', 'VERB'),
 ('Continued', 'VERB'),
 ('securities-based', 'VERB'),
 ('missed', 'VERB'),
 ('evolved', 'VERB'),
 ('relied', 'VERB'),
 ('bombarding', 'VERB'),
 ('signaling', 'VERB'),
 ('muscling', 'VERB'),
 ('swapping', 'VERB'),
 ('fawning', 'VERB'),
 ('wanting', 'VERB'),
 ('imposing', 'VERB'),
 ('Advancing', 'VERB'),
 ('insinuating', 'VERB'),
 ('injecting', 'VERB'),
 ('crying', 'VERB'),
 ('self-serving', 'VERB'),
 ('complaining', 'VERB'),
 ('pulling', 'VERB'),
 ('smother

In [85]:
T3 = list(set([pair[1] for pair in tagged_unknown_word_validation]))
T3

['ADJ', 'VERB', 'NOUN', 'NUM', 'ADV']

In [86]:
# compute word given tag, P(W/T)
def uwd_given_tag(word, tag, unknown_bag = tagged_unknown_word_validation):
    tag_list1 = [pair for pair in unknown_bag if pair[1] == tag]
    count_tag1 = len(tag_list1)
    w_given_tag_list1 = [pair[0] for pair in tag_list1 if pair[0] == word]
    count_w_given_tag1 = len(w_given_tag_list1)
    return (count_w_given_tag1, count_tag1)   

In [87]:
# Viterbi Heuristic for known common words in training set and validation set tagged with nltk tags in train_set and unknowns 
# words in validation set tagged as 'tagged_unknown_word_validation'

def Viterbi_improved2(words, train_bag = train_tagged_words, unknown_bag = tagged_unknown_word_validation):
    state = []
    T1 = list(set([pair[1] for pair in train_bag]))  # for tags in train_bag
    T2 = list(set([pair[0] for pair in train_bag]))  # for words in train_bag
    T3 = list(set([pair[1] for pair in unknown_bag]))  # for tags in unknown_bag
    T4 = list(set([pair[0] for pair in unknown_bag]))  # for words in unknown_bag
    
    for key, word in enumerate(words):        
        if word in T2:            
            #initialise list of probability column for a given observation
            p1 = [] 
            for tag in T1:
                if key == 0:
                    transition_p1 = tags_df.loc['.', tag]
                else:
                    transition_p1 = tags_df.loc[state[-1], tag]
                
                # compute emission and state probabilities
                emission_p1 = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability1 = emission_p1 * transition_p1    
                p1.append(state_probability1)
            
            p1max = max(p1)
            # getting state for which probability is maximum
            state_max1 = T1[p1.index(p1max)] 
            state.append(state_max1)
        elif word in T4:            
            #initialise list of probability column for a given observation
            p2 = [] 
            for tag in T3:                
                if key == 0:
                    transition_p2 = tags_df.loc['.', tag]
                else:
                    transition_p2 = tags_df.loc[state[-1], tag]
                
                # compute emission and state probabilities
                emission_p2 = uwd_given_tag(words[key], tag)[0]/uwd_given_tag(words[key], tag)[1]
                state_probability2 = emission_p2 * transition_p2    
                p2.append(state_probability2)
            
            p2max = max(p2)
            # getting state for which probability is maximum
            state_max2 = T3[p2.index(p2max)] 
            state.append(state_max2)    

    return list(zip(words, state))

In [88]:
start = time.time()
tagged_seq_improved2 = Viterbi_improved2(test_tagged_words)
end = time.time()
difference = end-start

In [89]:
print("Time taken in seconds: ", difference)
print(tagged_seq_improved2)

Time taken in seconds:  1084.6060483455658
[('Norman', 'NOUN'), ('Ricken', 'NOUN'), (',', '.'), ('52', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('president', 'NOUN'), ('and', 'CONJ'), ('chief', 'NOUN'), ('operating', 'NOUN'), ('officer', 'NOUN'), ('of', 'ADP'), ('Toys', 'NOUN'), ('``', '.'), ('R', 'NOUN'), ("''", '.'), ('Us', 'NOUN'), ('Inc.', 'NOUN'), (',', '.'), ('and', 'CONJ'), ('Frederick', 'NOUN'), ('Deane', 'NOUN'), ('Jr.', 'NOUN'), (',', '.'), ('63', 'NUM'), (',', '.'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Signet', 'NOUN'), ('Banking', 'NOUN'), ('Corp.', 'NOUN'), (',', '.'), ('were', 'VERB'), ('elected', 'VERB'), ('*-15', 'X'), ('directors', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('consumer', 'NOUN'), ('electronics', 'NOUN'), ('and', 'CONJ'), ('appliances', 'NOUN'), ('retailing', 'NOUN'), ('chain', 'NOUN'), ('.', '.'), ('The', 'DET'), ('court', 'NOUN'), ('hearing', 'NOUN'), ('began', 'VERB'), ('in', 'ADP'), ('early', 'ADJ'), ('October', 

In [90]:
# accuracy with improved2 Viterbi
check_improved2 = [i for i, j in zip(tagged_seq_improved2, test_run_base) if i == j] 

In [91]:
accuracy_improved2 = len(check_improved2)/len(tagged_seq_improved2)

In [92]:
round(accuracy_improved2*100,2)

96.32

In [93]:
incorrect_tagged_cases_2 = [[test_run_base[i-1], j] for i, j in enumerate(zip(tagged_seq_improved2, test_run_base)) if j[0]!=j[1]]

In [94]:
# Percentage of incorrect tags now with improved viterbi
round(len(incorrect_tagged_cases_2)/len(tagged_seq_improved2)*100,2)

3.68

#### Thus with solution-2 rule based improvement in Viterbi algorithm, tagging accuracy increased from 92% (for original viterbi) to 96.4%  on validation set

### Tagging accuracy check of the sample test sentences (provided in the assignment) with 2nd improved Viterbi

Here unknown bag for sample test sentences to be created.

In [95]:
# vocabulary of unique words in sample_set
V_sample_test = set(words_sample_test)
len(V_sample_test)

115

In [96]:
list(V_sample_test)

['smartphones',
 'firehose',
 'about',
 'ICESAT-2',
 'that',
 'FIFA',
 'mobile',
 'as',
 "'s",
 'be',
 'price',
 'media',
 'time',
 'interact',
 'This',
 'gave',
 'cheapest',
 'networking',
 'a',
 'OS',
 'since',
 'in',
 'post',
 '.',
 '11th',
 'news',
 'Android',
 'developed',
 'and',
 'to',
 'I',
 'arriving',
 'Google',
 'Trump',
 '2015',
 'was',
 'NASA',
 'Cup',
 'Europe',
 'Dallas',
 'The',
 'operating',
 '2018',
 'with',
 'held',
 'has',
 'years',
 'trips',
 'is',
 'Francisco',
 'at',
 'tweets',
 'an',
 'invited',
 'Twitter',
 'the',
 'would',
 '2011',
 'worldwide',
 'businessman',
 'been',
 'users',
 'on',
 'of',
 'system',
 'contested',
 'it',
 'from',
 'football',
 'access',
 'online',
 'San',
 'World',
 'known',
 'first',
 'every',
 '3',
 'see',
 'launch',
 'Eastern',
 'flights',
 'entering',
 'service',
 'me',
 'once',
 'best-selling',
 'like',
 'Satellite',
 'politics',
 'international',
 'made',
 'Show',
 'four',
 'messages',
 'personality',
 'tournament',
 'round',
 'leavi

In [97]:
# for unknown words in sample test set
unknown_word_sample = list_diff(V_sample_test, V)
unknown_word_sample

['smartphones',
 'worldwide',
 'Android',
 'messages',
 'personality',
 'firehose',
 'tournament',
 'ICESAT-2',
 'arriving',
 'Google',
 'contested',
 '2015',
 'NASA',
 'Cup',
 'online',
 '2013',
 'FIFA',
 '2018',
 'trips',
 'interact',
 'domineering',
 'Satellite',
 'tweets',
 'invited',
 'Twitter',
 '2011',
 'OS',
 '21st']

In [98]:
len(unknown_word_sample) # number of unknown words in sample test set 

28

In [99]:
# from this unknown words list, I will separate out words ending with 'ed', 'ing', 's', 'es' having morphological features
ed_verbs_sm = [word for word in unknown_word_sample if word.endswith('ed')]
ing_verbs_sm = [word for word in unknown_word_sample if word.endswith('ing')]
s_verbs_sm = [word for word in unknown_word_sample if word.endswith('s')]
es_verbs_sm = [word for word in unknown_word_sample if word.endswith('es')]

In [100]:
# list of morphological unknown words
morph_unknown_word_sample = ed_verbs_sm + ing_verbs_sm + s_verbs_sm + es_verbs_sm

In [101]:
morph_unknown_word_sample[:5]

['contested', 'invited', 'arriving', 'domineering', 'smartphones']

In [102]:
len(morph_unknown_word_sample)

10

In [103]:
remain_unknown_word_sample = list_diff(set(unknown_word_sample), set(morph_unknown_word_sample))

In [104]:
# tagging remaining unknown words in sample set with nltk universal tag sets
tagged_remain_unknown_words_sample = nltk.pos_tag(remain_unknown_word_sample, tagset='universal')

In [105]:
tagged_remain_unknown_words_sample

[('worldwide', 'ADV'),
 ('Android', 'NOUN'),
 ('personality', 'NOUN'),
 ('firehose', 'ADJ'),
 ('tournament', 'ADJ'),
 ('ICESAT-2', 'NOUN'),
 ('Google', 'NOUN'),
 ('2015', 'NUM'),
 ('NASA', 'NOUN'),
 ('Cup', 'NOUN'),
 ('online', 'NOUN'),
 ('2013', 'NUM'),
 ('FIFA', 'NOUN'),
 ('2018', 'NUM'),
 ('interact', 'NOUN'),
 ('Satellite', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('2011', 'NUM'),
 ('OS', 'NOUN'),
 ('21st', 'NUM')]

In [106]:
total_tags_remain_uwds1 = [pair[1] for pair in tagged_remain_unknown_words_sample] # uwds1 stand for remaining unknown words in sample test

In [107]:
tag_counts_remain_uwds1 = Counter(total_tags_remain_uwds1)

In [109]:
tag_counts_remain_uwds1.most_common(3)

[('NOUN', 12), ('NUM', 5), ('ADJ', 2)]

most of the words are tagged with Noun already 

In [110]:
tagged_morph_unknown_word_sample = rule_based_tagger.tag(morph_unknown_word_sample)
tagged_morph_unknown_word_sample[:3]

[('contested', 'VERB'), ('invited', 'VERB'), ('arriving', 'VERB')]

In [111]:
tagged_unknown_word_sample = tagged_morph_unknown_word_sample + tagged_remain_unknown_words_sample
tagged_unknown_word_sample

[('contested', 'VERB'),
 ('invited', 'VERB'),
 ('arriving', 'VERB'),
 ('domineering', 'VERB'),
 ('smartphones', 'NOUN'),
 ('messages', 'NOUN'),
 ('trips', 'NOUN'),
 ('tweets', 'NOUN'),
 ('smartphones', 'NOUN'),
 ('messages', 'NOUN'),
 ('worldwide', 'ADV'),
 ('Android', 'NOUN'),
 ('personality', 'NOUN'),
 ('firehose', 'ADJ'),
 ('tournament', 'ADJ'),
 ('ICESAT-2', 'NOUN'),
 ('Google', 'NOUN'),
 ('2015', 'NUM'),
 ('NASA', 'NOUN'),
 ('Cup', 'NOUN'),
 ('online', 'NOUN'),
 ('2013', 'NUM'),
 ('FIFA', 'NOUN'),
 ('2018', 'NUM'),
 ('interact', 'NOUN'),
 ('Satellite', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('2011', 'NUM'),
 ('OS', 'NOUN'),
 ('21st', 'NUM')]

In [112]:
T5 = list(set([pair[1] for pair in tagged_unknown_word_sample]))
T5

['ADJ', 'VERB', 'NOUN', 'NUM', 'ADV']

In [113]:
# compute word given tag, P(W/T)
def uwd1_given_tag(word, tag, unknown_bag1 = tagged_unknown_word_sample):  #unknown_bag1 is for sample test sentences
    tag_list2 = [pair for pair in unknown_bag1 if pair[1] == tag]
    count_tag2 = len(tag_list2)
    w_given_tag_list2 = [pair[0] for pair in tag_list2 if pair[0] == word]
    count_w_given_tag2 = len(w_given_tag_list2)
    return (count_w_given_tag2, count_tag2)   

In [114]:
# Viterbi Heuristic for known common words in training set and validation set tagged with nltk tags in train_set and unknowns 
# words in sample test set tagged as 'tagged_unknown_word_sample'

def Viterbi_improved2_(words, train_bag = train_tagged_words, unknown_bag1 = tagged_unknown_word_sample):
    state = []
    T1 = list(set([pair[1] for pair in train_bag]))  # for tags in train_bag
    T2 = list(set([pair[0] for pair in train_bag]))  # for words in train_bag
    T5 = list(set([pair[1] for pair in unknown_bag1]))  # for tags in unknown_bag1
    T6 = list(set([pair[0] for pair in unknown_bag1]))  # for words in unknown_bag1
    
    for key, word in enumerate(words):        
        if word in T2:            
            #initialise list of probability column for a given observation
            p1 = [] 
            for tag in T1:
                if key == 0:
                    transition_p1 = tags_df.loc['.', tag]
                else:
                    transition_p1 = tags_df.loc[state[-1], tag]
                
                # compute emission and state probabilities
                emission_p1 = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
                state_probability1 = emission_p1 * transition_p1    
                p1.append(state_probability1)
            
            p1max = max(p1)
            # getting state for which probability is maximum
            state_max1 = T1[p1.index(p1max)] 
            state.append(state_max1)
        elif word in T6:            
            #initialise list of probability column for a given observation
            p3 = [] 
            for tag in T5:                
                if key == 0:
                    transition_p3 = tags_df.loc['.', tag]
                else:
                    transition_p3 = tags_df.loc[state[-1], tag]
                
                # compute emission and state probabilities
                emission_p3 = uwd1_given_tag(words[key], tag)[0]/uwd1_given_tag(words[key], tag)[1]
                state_probability3 = emission_p3 * transition_p3    
                p3.append(state_probability3)
            
            p3max = max(p3)
            # getting state for which probability is maximum
            state_max3 = T5[p3.index(p3max)] 
            state.append(state_max3)    

    return list(zip(words, state))

In [115]:
# 2nd improved Viterbi tagging for words in sample test sentences

start = time.time()
viterbi2_tagged_sample_test = Viterbi_improved2_(words_sample_test)
end = time.time()
difference = end-start

In [116]:
print("Time taken in seconds: ", difference)
print(viterbi2_tagged_sample_test)

Time taken in seconds:  32.65073561668396
[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'ADV'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'ADJ'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NO

In [117]:
# accuracy with 2nd improved Viterbi
check_vtb2 = [i for i, j in zip(viterbi2_tagged_sample_test, tagged_sample_test) if i == j] 

In [118]:
accuracy_vtb2 = len(check_vtb2)/len(viterbi2_tagged_sample_test)

In [120]:
round(accuracy_vtb2*100,2)

92.82

#### Thus with 2nd improved viterbi, tagging accuracy increased from 77.4% (for original viterbi) to 93% on the sample test sentences.

List of few words having tagging with Vanilla viterbi vs. 2nd improved vitrbi -

1. ('Android', 'ADJ') with Vanilla, ('Android', 'NOUN') with 2n improved
2. ('Google', 'ADJ') with Vanilla, ('Google', 'NOUN') with 2nd improved
3. ('smartphones', 'ADJ') with Vanilla, ('smartphones', 'NOUN') with 2nd improved
4. ('Twitter', 'ADJ') with Vanilla, ('Twitter', 'NOUN') with 2nd improved
5. ('online', 'ADJ') with Vanilla, ('online', 'NOUN') with 2nd improved
6. ('messages', 'ADJ') with Vanilla, ('messages', 'NOUN') with 2nd improved
7. ('tweets', 'ADJ') with Vanilla, ('tweets', 'NOUN') with 2nd improved
8. ('2011', 'ADJ') with Vanilla, ('2011', 'NUM') with 2nd improved
9. ('2013', 'ADJ') with Vanilla, ('2013', 'NUM') with 2nd improved
10. ('2015', 'ADJ') with Vanilla, ('2015', 'NUM') with 2nd improved
11. ('2018', 'ADJ') with Vanilla, ('2018', 'NUM') with 2nd improved
12. ('21st', 'ADJ') with Vanilla, ('21st', 'NUM') with 2nd improved
13. ('FIFA', 'ADJ') with Vanilla, ('FIFA', 'NOUN') with 2nd improved

### End of Section-2

## Conclusion-1: 
Comparison of the tagging accuracies for the modifications with the vanilla Viterbi algorithm

1. Tagging accuracy with vanilla viterbi algorithm on the validation set was found as 92%
2. Tagging accuracy with 1st modified viterbi algorithm on the validation set was found as 95%
3. Tagging accuracy with 2nd modified viterbi algorithm on the validation set was found as 96.4%

4. Tagging accuracy with vanilla viterbi algorithm on the sample test sentences was found as 77.4%
5. Tagging accuracy with 1st modified viterbi algorithm on the sample test sentences was found as 89.5%
6. Tagging accuracy with 2nd modified viterbi algorithm on the sample test sentences was found as 93%.  

## Conclusion-2: 
Cases which were incorrectly tagged by original POS tagger and got corrected by my modifications:

Incorrectly tagged few words by original Vanilla POS tagger - 

1. ('Android', 'ADJ') 
2. ('Google', 'ADJ') 
3. ('smartphones', 'ADJ')
4. ('Twitter', 'ADJ') 
5. ('online', 'ADJ')
6. ('messages', 'ADJ')
7. ('tweets', 'ADJ')
8. ('2011', 'ADJ')
9. ('2013', 'ADJ')
10. ('2015', 'ADJ')
11. ('2018', 'ADJ')
12. ('21st', 'ADJ'
13. ('FIFA', 'ADJ')

Correctly tagged same above words by 1st modification POS tagger - 

1. ('Android', 'NOUN') 
2. ('Google', 'NOUN') 
3. ('smartphones', 'NOUN')
4. ('Twitter', 'NOUN') 
5. ('online', 'NOUN')
6. ('messages', 'NOUN')
7. ('tweets', 'NOUN')
13. ('FIFA', 'NOUN') 

Correctly tagged same above words by 2nd modification POS tagger - 

1. ('Android', 'NOUN') 
2. ('Google', 'NOUN') 
3. ('smartphones', 'NOUN')
4. ('Twitter', 'NOUN') 
5. ('online', 'NOUN')
6. ('messages', 'NOUN')
7. ('tweets', 'NOUN')
8. ('2011', 'NUM')
9. ('2013', 'NUM')
10. ('2015', 'NUM')
11. ('2018', 'NUM')
12. ('21st', 'NUM')
13. ('FIFA', 'NOUN')
